In [ ]:
from sklearn.datasets import load_iris, fetch_openml
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

# Einstieg ins maschinelle Lernen

## Datensatz: Schwertlilien

Für unseren Einstieg ins klassische maschinelle Lernen verwenden wir den **Iris**-Datensatz, den Sie aus der ersten Übung bereits kennen. Zunächst laden wir den Datensatz, der im wesentlichen aus Merkmalsmatrix $\mathbf{X}$ und Zielvektor $\mathbf{y}$ besteht. Wie in der Statistik üblich bezeichnen wir die Spalten der Merkmalsmatrix hier auch als **Variablen**, die für jedes Iris-Exemplar den entsprechenden Merkmalswert annehmen.

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target

Für überwachtes Lernen müssen wir den Datensatz in Trainings- und Testdaten aufteilen (hier je 50%). Ein Development-Set ist nicht erforderlich, da wir die Optimierung der Metaparameter später per Kreuzvalidierung vornehmen (via `GridSearchCV`).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, stratify=y, random_state=42)

Wir überprüfen, dass der Datensatz stratifiziert aufgeteilt wurde, so dass in jedem Set genau 25 Exemplare jeder Iris-Art enthalten sind.

In [ ]:
pd.concat([
    pd.Series(y_train).value_counts(), 
    pd.Series(y_test).value_counts()
], axis = 1)

> **Frage:** Warum ist es sinnvoll, die Merkmalsvektoren wie in der ersten Übung zu standardisieren? Was passiert bei der Standardisierung genau? (Tipp: Denken Sie an Regularisierungsfunktionen.)

Als scikit-learn-Expert:innen bauen wir die Standardisierung natürlich in eine `Pipeline` ein, statt die Merkmalsmatrix vorab zu modifizieren.

## Lineare Klassifikation

Wir trainieren und evaluieren nun einen lineares Klassifikator.  Als Standardverfahren bietet sich **Logistic Regression** an, die wir in einigen Wochen auch selbst mit Hilfe von NumPy implementieren werden. Im Gegensatz zu unserer eigenen Implementierung unterstützt Scikit-Learn auch multinomiale Klassifikationsprobleme mit mehr als zwei Kategorien.

Wir erstellen eine Pipeline mit zwei Komponenten: Standardisierung (`StandardScaler`) und Klassifikation (`LogisticRegression`). Eine Komponente zur Merkmalsextraktion benötigen wir nicht, da als Eingabe ja schon eine numerische Merkmalsmatrix vorliegt. Die Metaparameter der Verfahren belassen wir zunächst bei den voreingestellten Werten.

In [ ]:
pipe = Pipeline([
    ('std', StandardScaler()),
    ('clf', LogisticRegression())
])

Nun können wir die Pipeline trainieren und die erreichte Genauigkeit auf den Trainingsdaten überprüfen.

In [ ]:
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train)

Wie erwartet erzielt das Klassifikationsverfahren auf dem kleinen Trainingsdatensatz fast perfekte Ergebnisse: nur 2 von 75 Iris-Exemplaren werden falsch klassifiziert. Interessanter ist natürlich die Evaluation auf den Testdaten.

In [ ]:
pred_test = pipe.predict(X_test)
print(classification_report(y_test, pred_test, target_names=iris.target_names))

Offenbar liegt keine starke Überanpassung vor: die Ergebnisse auf den Testdaten sind nur unwesentlich schlechter. _Iris setosa_ wird dabei perfekt erkannt, nur bei der Abgrenzung zwischen _Iris versicolor_ und _Iris virginica_ kommt es zu ingesamt 4 Fehlern.

> **Frage:** Können Sie das Evaluationsergebnis erklären, wenn Sie sich die Visualisierung aus der ersten Übung anschauen?

Die Parameter eines linearen Klassifikators können intuitiv als **Merkmalsgewichte** interpretiert werden (im Gegensatz zu den meisten komplexeren Lernverfahren, die oft schwer anschaulich zu interpretieren sind). 

In [ ]:
W = pipe['clf'].coef_
b = pipe['clf'].intercept_

print(W.round(3))
print()
print(b.round(3))

> **Frage:** Wofür stehen $\mathbf{W}$ und $\mathbf{b}$? Ordnen Sie die Merkmalsgewichte den Kategorien und Variablen zu. Können Sie die Gewichte anschaulich interpretieren? (Tipp: schauen Sie sich dazu wieder die Visualisierungen aus der ersten Übung an.)

## Tuning

> **Aufgabe:** Für gute Lernergebnisse ist es in den allermeisten Fällen wichtig, die Metaparameter der Lernverfahren (und auch der Merkmalsextraktion und anderer Vorverarbeitungsschritte) systematisch zu optimieren. Erinnern Sie sich, wie Sie zu diesem Zweck eine systematische Grid Search durchführen können? Welche Metaparameter könnten hier von Interesse sein?

Wir evaluieren das beste Modell noch auf den Testdaten, um zu überprüfen, ob das Tuning tatsächlich zu einer Verbesserung geführt oder nur die Überanpassung erhöht hat. Leider scheint eher letzteres der Fall zu sein.

In [ ]:
pipe_tuned = gs.best_estimator_
pred_test = pipe_tuned.predict(X_test)
print(classification_report(y_test, pred_test, target_names=iris.target_names))

> **Aufgabe:** Probieren Sie auch andere Klassifikationsverfahren aus (z.B. SVM, SGD, Nearest Neighbour, Decision Tree). Lesen Sie dazu jeweils die Dokumentation der entsprechenden scikit-learn-Klasse unter https://scikit-learn.org/stable/user_guide.html. Wie gut funktionieren diese anderen Lernverfahren? Können Sie deren Parameter auch anschaulich interpretieren?

# Ziffernerkennung

## Datensatz: handgeschriebene Ziffern (MNIST)

Der MNIST-Datensatz ist ein klassischer (aber ziemlich einfacher) „Benchmark“ der Bildverarbeitung, bei dem es speziell um die Erkennung handgeschriebener Ziffern geht (z.B. für Postleitzahlen oder Überweisungsformulare). Der Datensatz besteht aus je 7.000 Bildern der Ziffern 0, …, 9 und kann leicht mit Hilfe von Scikit-Learn geladen werden.

In [ ]:
mnist = fetch_openml("mnist_784", as_frame=False)
mnist.data.shape

Die Bilder liegen haben ein Format von $28\times 28$ Pixeln, die aber als „flache“ Vektoren $\mathbf{x}\in \mathbb{R}^{784}$ gespeichert sind. Wenn wir diese Vektoren als $28\times 28$-Matrix ausgeben, sind die Ziffern erkennbar

In [ ]:
np.set_printoptions(linewidth=160)
print(mnist.data[7, :].reshape((28, 28)))

Die Pixel werden als Graustufenwerte von 0 bis 255 gespeichert. Statt die einzelnen Pixel (als Variablen der Merkmalsvektoren) zu standardisieren, können wir sie auf den ähnlichen Bereich $[0, 1]$ umskalieren. Damit sollten die meisten Lernverfahren gut umgehen können. Tatsächlich führt die ursprüngliche Skalierung zu Problemen mit der Regularisierung einiger Lernverfahren, die dann nur mit sehr unüblichen Regularisierungsparametern gut funktionieren.

Die Zielkategorien sind hier als Zeichenketten `'0'` bis `'9'` kodiert. Wir konvertieren sie in ganzzahlige Werte, wie von den multinomialen Klassifikationsverfahren in Scikit-Learn erwartet wird. Da `fetch_openml()` leider keine Kategorienlabel zurückliefert, legen wir diese selbst in der Variablen `cat_names` an.

In [ ]:
X = mnist.data / 255
y = mnist.target.astype('int')
cat_names = [str(x) for x in range(10)]

Um einen bessern Eindruck von der Aufgabenstellung zu bekommen, definieren wir eine Hilfsfunktion, die die Merkmalsvektoren als Pixelbilder anzeigt. Grundsätzlich verwenden wir dazu `plt.imgshow()` mit `cmap="binary"` für Schwarzweißbilder. Um mehrere Ziffern in einer Graphik darzustellen, fasst `mk_imagemap()` diese in eine große Matrix zusammen. Als Beispiel zeigen wir die ersten 50 Ziffern aus dem Datensatz an. Auf die zugehörigen Goldstandard-Label können wir ganz offensichtlich verzichten.

In [ ]:
def mk_imagemap(data, nrow, ncol, padding=2):
    w, h = data.shape[-2:]
    data = data.reshape((-1, w, h))
    n = data.shape[0]
    image = np.zeros((nrow * h + (nrow - 1) * padding, ncol * w + (ncol - 1) * padding))
    y = 0
    k = 0
    for i in range(nrow):
        x = 0
        for j in range(ncol):
            if k <= n - 1:
                image[y:y+h, x:x+w] = data[k]
            x += w + padding
            k += 1
        y += h + padding
    return image

plt.imshow(mk_imagemap(X.reshape((70000, 28, 28)), 5, 10), cmap="binary");

Wir teilen den MNIST-Datensatz in 80% Trainingsdaten und 20% Testdaten auf (was aufgrund des großen Gesamtumfangs völlig ausreichend ist).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print(X_train.shape)
print(X_test.shape)

## Lineare Klassifikation

Wir beginnen wieder mit einem einfachen linearen Klassifikationsverfahren. Zur Wahl stehen neben der bisher verwendeten `LogisticRegression` unter anderem auch Support Vector Machines (nur die lineare Variante `LinearSVC` ist effizient genug) und Stochastic Gradient Descent (`SGDClassifier`). Mit letzterem werden wir uns in den nächsten Wochen noch ausführlich beschäftigen, da es eine der zentralen Grundlagen für Deep Learning darstellt.

Beachten Sie, dass die in Scikit-Learn implementierten Lernverfahren die Bilder einfach als „flache“ Merkmalsvektoren verarbeiten und ihre zweidimensionale Struktur nicht ausnutzen können. Das wird erst später mit Hilfe spezieller Deep-Learning-Modelle möglich.

Da wir es jetzt mit einer erheblich größeren Menge von Trainingsdaten und höherdimensionalen Merkmalsvektoren zu tun haben, dauert die Parameterschätzung der Lernverfahren (durch Minimierung der Kostenfunktion $J(\mathbf{w}; T) = L(\mathbf{w}; T) + \lambda C(\mathbf{w})$) wesentlich länger als bisher. Mit der Direktive `%%time` am Anfang der Zelle können wir jeweils benötigte Rechenzeit anzeigen lassen. Für die logistische Regression müssen Sie mit einer Trainingsdauer von deutlich über 1 Minute rechnen.

In [ ]:
%%time
clf = LogisticRegression(C=1, max_iter=1000)
clf.fit(X_train, y_train)
print(f"Training accuracy: {clf.score(X_train, y_train):.4f}")

Entscheidend ist natürlich wieder die Evaluation auf den Testdaten. Wie üblich verwenden wir hierzu `classification_report()`, um auch Precision und Recall für jede einzelne Kategorie zu bestimmen.

In [ ]:
pred_test = clf.predict(X_test)
print(classification_report(y_test, pred_test, digits=4, target_names=cat_names))

> **Frage:** Wie beurteilen Sie die Evaluationsergebnisse? Schauen Sie sich insbesondere auch Precision und Recall für die einzelnen Ziffern an. Gibt es hier auffällige Unterschiede? Wenn ja: was könnten die Ursachen dafür sein?

> **Aufgabe:** Ein besseres Verständnis der Klassifikationsfehler lässt sich oft aus der _confusion matrix_ ableiten. Erstellen Sie eine solche Fehlermatrix für unseren Ziffern-Klassifikator. Können Sie die Matrix mit Hilfe von `ConfusionMatrixDisplay` auch visualisieren?

## Interpretation der Merkmalsgewichte

Die Interpretation der Parameter eines linearen Klassifikators als Merkmalsgewichte ist in diesem Fall besonders anschaulich. Es handelt sich nämlich (jeweils pro Kategorie = Ziffer) um Gewichtungen für die einzelnen Pixel der Bilder. Positive Gewichte markieren Pixel, die _für_ die jeweilige Ziffer sprechen; negative Gewichte markieren Pixel, die _gegen_ die Ziffer sprechen (also bei dieser Ziffer meistens nicht schwarz sind). Wir können die Gewichte also auch als $28\times 28$-Bilder visualisieren, indem wir die Gewichtsvektoren $\mathbf{w}\in \mathbb{R}^{784}$ jeweils in eine quadratische Matrix $\in \mathbb{R}^{28\times 28}$ umformen.

Zu diesem Zweck definieren wir eine kleine Hilfsfunktion, die `mk_imagemap()` zur Darstellung der 10 Gewichtsmatrizen nutzt. Entscheidend ist hierbei, den Wertebereich der Pixel so einzustellen, dass ein Gewicht von 0 genau in der Mitte liegt. (**Frage:** Warum ist das wichtig? Wie wird es in unserer Hilfsfunktion sichergestellt?)

In [ ]:
def plot_weights(W, cmap='bwr', vmax=None):
    image = mk_imagemap(W.reshape((-1, 28, 28)), 2, 5)
    if vmax is None:
        vmax = np.abs(image).max()
        print(f"range: [{-vmax:.2f}, {vmax:.2f}]")
    plt.imshow(image, cmap=cmap, vmin=-vmax, vmax=vmax)

plot_weights(clf.coef_)

Rot steht hier für positive Gewichte, blaue für negative, weiß für das Gewicht 0 (also Pixel, die vom Klassifikator gar nicht berücksichtigt werden). Der Farbton ist umso intensiver, je größer die Gewichte sind. Da einige Pixel sehr große Merkmalsgewichte erhalten, bleiben die Farbtöne der meisten anderen Pixel relativ schwach. Wir können mit dem Parameter `vmax=` die Darstellungsskala anpassen, um auch die mittelstarken Gewichte deutlicher zu visualisieren. Dann ist allerdings keine Unterscheidung zwischen großen und sehr großen Gewichten mehr möglich. 

In [ ]:
plot_weights(clf.coef_, vmax=2)

Mit etwas Fanatsie sind zumindest in einigen Fällen (z.B. `0`, `2`, `3`, `8`) die groben Formen der Ziffern in rot auszumachen. Negative Gewichte füllen oft Zwischenräume aus und helfen so bei der Abgrenzung von ähnlichen Ziffern. Markant ist dies z.B. bei der `0`, die in der Bildmitte einen leeren Bereich haben muss. Bei einigen anderen Ziffern (u.a. `1`, `4`, `7`) lässt sich kaum eine vertraute Form erkennen. Auch das erklärt sich schnell durch einen Blick auf die Beispielbilder: für diese Ziffern gibt es viele unterschiedliche Schreibweisen und Orientierungen, die alle in einen einzigen Gewichtsvektor kombiniert werden müssen. Grundsätzlich haben daher alle Pixel, die in _irgendeiner_ Variante vorkommen, positive Gewichte; nur Pixel, die in keiner der Varianten auftauchen, haben negative Gewichte.

> **Aufgabe:** Experimentieren Sie mit der Regularisierungsstärke und anderen Metaparameter, oder probieren Sie andere lineare Klassifikationsverfahren aus. (Da die einzelnen Trainingsdurchläufe recht langwierig sind, verzichten wir auf ein systematisches Tuning der Metaparameter.) Wie gut fallen die Ergebnisse aus? Wie veränderen sich die Pixelgewichte? Es bietet sich an, eine kleine Hilfsfunktion zur Evaluation des trainierten Klassifikators und der Visualisierung der Merkmalsgewichte zu definieren.

In [ ]:
def eval_linclf(clf, on_train=True, confusion=False, plot=True, vmax=None):
    if on_train:
        print(f"Training accuracy: {clf.score(X_train, y_train):.4f}")
    pred_test = clf.predict(X_test)
    print(classification_report(y_test, pred_test, digits=4, target_names=cat_names))
    if confusion:
        print(confusion_matrix(y_test, pred_test))
    if plot:
        plot_weights(clf.coef_, vmax=vmax)

In [ ]:
%%time


## Stochastic Gradient Descent

In der Vorlesung haben wir Stochastic Gradient Descent bereits als ein effizientes Verfahrung zur Parameteroptimierung linearer Klassifikatoren und insbesondere der logistischen Regression kennengelernt. Hier wird die Regularisierungsstärke direkt über den Parameter `alpha` bestimmt (entsprechend unserem $\lambda$). Die Lernrate wird in der Voreinstellung automatisch angepasst. Mit der Einstellung `n_jobs = -1` können wir das Training über alle verfügbaren CPU-Kerne parallelisieren.

In [ ]:
%%time
clf = SGDClassifier(alpha=1e-3, max_iter=5000, n_jobs=-1)
clf.fit(X_train, y_train)

In [ ]:
eval_linclf(clf)

In kürzerster Zeit erreichen wir so ein Ergebnis, das nur wenig hinter logistischer Regression und SVM zurückbleibt (mit immer noch deutlich über 90% Genauigkeit auf den Testdaten). Die Merkmalsgewichte sind deutlich besser zu interpretieren als bei `LogisticRegression` und lassen die meisten Ziffern gut erkennen.

> **Aufgabe:** Was passiert, wenn Sie die Regularisierungsstärke $\alpha$ erhöhen? Wie verändern sich Genauigkeit, Zeitaufwand und Merkmalsgewichte? Was ist, wenn Sie statt $L_2$-Regularisierung eine $L_1$-Regularisierung verwenden (`penalty`)? Versuchen Sie auch manuell die Lernrate $\eta$ (`eta0`) einzustellen, wofür Sie `learning_rate='constant'` oder `learning_rate='adaptive'` auswählen müssen.

## Nichtlineare Klassifikation

Unsere bisherigen Erkenntnisse legen nahe, dass lineare Klassifikationsverfahren für die Ziffernerkennung nur bedingt geeignet sind. Insbesondere können sie verschiedene Schreibvarianten der gleichen Ziffer nicht separat modellieren. Wir wenden uns daher im letzten Abschnitt nichtlinearen Lernverfahren zu.

Ein Standardverfahren für nichtlineare Klassifikation sind SVM, die sich durch Auswahl eines geeigneten _kernel_ flexibel konfigurieren lassen. In unserem Fall dürften polynomiale Kerne wenig Vorteile bringen – ein kubischer Kern kann z.B. bestenfalls Kombinationen von je drei Pixeln berücksichtigen. Am flexibelsten ist der RBF-Kern, der aber sowohl beim Training als auch bei der Anwendung auf die Testdaten sehr langsam arbeitet.  Führen Sie die nächsten beiden Zellen am besten vor einer Kaffeepause aus …

In [ ]:
%%time
rbf = SVC(kernel='rbf', C=1.0)
rbf.fit(X_train, y_train)

In [ ]:
%%time
eval_linclf(rbf, on_train=False, plot=False)

Die SVM mit RBF-Kern erzielt erheblich bessere Ergebnisse als die linearen Klassifikationsverfahren und erreicht auch ohne Tuning eine Genauigkeit von nahezu 98%. Allerdings gibt es hier keine interpretierbaren Merkmalsgewichte. 

Es gibt auch einige andere Klassifikationsverfahren, die nichtlineare Muster lernen können. Dazu gehören Entscheidungsbäume (die aber in jedem Schritt nur einen einzelnen Pixel berücksichtigen können und daher für die Ziffernerkennung eher nicht geeignet sind) und Nearest-Neighbour-Verfahren. Ein `DecisionTreeClassifier` ist schnell trainiert, zeigt aber extreme Überanpassung an die Trainingsdaten und enttäuschende Evaluationsergebnisse.

In [ ]:
%%time
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

In [ ]:
eval_linclf(dt, plot=False)

Der gelernte Entscheidungsbaum ist sehr komplex und für Menschen sicher nicht verständlich:

In [ ]:
print("Insgesamt {} Entscheidungsknoten bei maximaler Tiefe von {} Schritten.".format(
    dt.tree_.node_count, dt.tree_.max_depth))

Bei unserem großen Datensatz und einer zufälligen Aufteilung in Trainings- und Testdaten bietet sich ein Nearest-Neighbour-Verfahren an. Es dürfte sehr gute Ergebnisse lieferen, so lange sich zu jedem Bild in den Testdaten eine sehr ähnlich geschrieben Ziffer in den Trainingsdaten findet. Entscheidenen Parameter sind die Anzahl der nächsten Nachbarn, die für die Entscheidung berücksichtigt werden, sowie das zu verwendende Abstandsmaß.

Ein Training im eigentlichen Sinn findet nicht statt: es werden leglich sämtliche Trainings-Datenpunkte abgespeichert. Dafür ist die Anwendung auf neue Daten sehr zeitaufwändig, da der Abstand von jedem neuen Merkmalsvektor zu allen Trainingsdaten berechnet werden muss. Insbesondere ist es wichtig, die langwierige und wenig interessante Evaluation auf den Trainingsdaten zu überspringen.

In [ ]:
nn = KNeighborsClassifier(n_neighbors=10, metric='manhattan')
nn.fit(X_train, y_train);

In [ ]:
%%time
eval_linclf(nn, on_train=False, plot=False)

Auch diese sehr simple Verfahren erzielt deutlich bessere Ergebnisse als die lineare Klassfikation.  Bessere nichtlineare Verfahren werden wir dann später mit speziellen Deep-Learning-Modellen erproben können.